In [3]:
#! ls data

In [9]:
from PIL import Image
a = Image.open('data/1.jpg')
import numpy as np
b = np.array(a)
b.resize(47,135)
b.shape

(47, 135)

In [11]:
height, width = 47,135

In [81]:
# 載入產生Capthca的函式庫
import matplotlib.pyplot as plt
import numpy as np
import random

import string

characters = '0123456789abcdefghijklmnopqrstuvwxyz'
# Captcha生成圖像的寬高、用幾個字母來組成字串及會用到的字母數
width, height, n_len, n_class = 135, 47, 4, 36


In [82]:
from keras.models import *
from keras.layers import *

input_tensor = Input((height, width, 3))
x = input_tensor

# 產生有四個 block的卷積網絡
#x = Conv2D(32, (3, 3), activation='relu')(x)
#x = Conv2D(32, (3, 3), activation='relu')(x)
#x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu')(x)
x = Conv2D(128, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x) 

x = Conv2D(256, (3, 3), activation='relu')(x)
x = Conv2D(256, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x) 

x = Flatten()(x)
x = Dropout(0.25)(x)



In [83]:
x = [Dense(n_class, activation='softmax', name='c%d'%(i+1))(x) for i in range(4)]

model = Model(inputs=input_tensor, outputs=x)

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 47, 135, 3)   0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 45, 133, 64)  1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 43, 131, 64)  36928       conv2d_7[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 21, 65, 64)   0           conv2d_8[0][0]                   
____________________________________________________________________________________________

In [84]:
import os 
os.mkdir('trainset')

FileExistsError: [Errno 17] File exists: 'trainset'

In [85]:
#os.mkdir('trainset/batch3')
import pandas
df = pandas.read_csv('output_captua.csv', index_col=0)
df.dropna(inplace=True)
df['file'] = df['file'].map(lambda e: 'data/{}.jpg'.format(int(e)) )

In [86]:
df.head()

,captcua,file
0.0,4ay2,data/1.jpg
2.0,qmqh,data/2.jpg
4.0,kc3c,data/3.jpg
6.0,1549,data/4.jpg
8.0,5rqz,data/5.jpg


In [87]:
df = df[df.index.notnull()]

In [88]:
len(df)

439

In [89]:
n_class

36

In [90]:
df.reset_index(inplace=True)
del df['index']

In [91]:
X = np.zeros((len(df), height, width, 3), dtype=np.float32)
y = [np.zeros((len(df), n_class), dtype=np.uint8) for i in range(n_len)]
for i in df.index:
    random_str = df.loc[i,'captcua']
    fname = df.loc[i,'file']
    f1 = Image.open(fname)
    f1 = f1.resize((width, height))
    f = np.array(f1)
    #print(f.shape)
    #f.resize(height,width)
    X[i] = f * 1.0 / 255.0
    for j, ch in enumerate(random_str):
        y[j][i, :] = 0
        y[j][i, characters.find(ch)] = 1

In [92]:
X.shape

(439, 47, 135, 3)

In [93]:
y[0][1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [94]:
X[0]

array([[[0.9254902 , 0.9254902 , 0.9254902 ],
        [0.7372549 , 0.7372549 , 0.7372549 ],
        [0.9254902 , 0.9254902 , 0.9254902 ],
        ...,
        [0.84705883, 0.84705883, 0.84705883],
        [0.8392157 , 0.8392157 , 0.8392157 ],
        [0.78431374, 0.78431374, 0.78431374]],

       [[0.827451  , 0.827451  , 0.827451  ],
        [0.9372549 , 0.9372549 , 0.9372549 ],
        [0.81960785, 0.81960785, 0.81960785],
        ...,
        [0.6392157 , 0.6392157 , 0.6392157 ],
        [0.69411767, 0.69411767, 0.69411767],
        [0.8       , 0.8       , 0.8       ]],

       [[0.9764706 , 0.9764706 , 0.9764706 ],
        [0.627451  , 0.627451  , 0.627451  ],
        [0.8392157 , 0.8392157 , 0.8392157 ],
        ...,
        [0.65882355, 0.65882355, 0.65882355],
        [0.73333335, 0.73333335, 0.73333335],
        [0.6862745 , 0.6862745 , 0.6862745 ]],

       ...,

       [[0.8784314 , 0.8784314 , 0.8784314 ],
        [0.7294118 , 0.7294118 , 0.7294118 ],
        [0.81960785, 0

In [95]:
import numpy 
ny = numpy.array(y)

In [96]:
ny[0][0]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [97]:
np.save('captcha_X3.npy', X)

In [98]:
np.save('captcha_y3.npy', ny)

In [99]:
ny[0][0]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

In [100]:
def gen(batch_size=32):
    X = np.zeros((batch_size, height, width, 3), dtype=np.uint8)
    y = [np.zeros((batch_size, n_class), dtype=np.uint8) for i in range(n_len)]
    while True:
        
        for i in range(batch_size):
            rand_index = random.choice(df.index)
            random_str = df.loc[rand_index,'captcua']
            fname = df.loc[rand_index,'file']
                #print('fname',fname,batchall.loc[rand_index,'path'], '123')
            #print(fname)
            f1 = Image.open(fname)
            f1 = f1.resize((width, height))
            f = np.array(f1)
            #print(f.shape)
            X[i] = f / 255.0

            for j, ch in enumerate(random_str):
                y[j][i, :] = 0
                y[j][i, characters.find(ch)] = 1
        yield X, y

In [101]:
gen(batch_size = 32)


<generator object gen at 0x13f326af0>

In [102]:
import keras
keras.models.load_model

<function keras.engine.saving.load_model>

In [103]:
from keras import callbacks
cbks = [callbacks.ModelCheckpoint("best_model.h5", save_best_only=True)]

history = model.fit_generator(gen(batch_size=32),      # 每次生成器會產生32筆小批量的資料
                    epochs=20,                # 總共跑5個訓練循環
                    steps_per_epoch=1000,
                    #callbacks=cbks,          # 保存最好的模型到檔案
                    validation_data=gen(),   # 驗證資料也是用生成器來產生
                    validation_steps=40      # 用40組資料來驗證
                   ) 

Epoch 1/20
 105/1000 [==>...........................] - ETA: 11:12 - loss: 10.9740 - c1_loss: 2.7515 - c2_loss: 2.7553 - c3_loss: 2.7597 - c4_loss: 2.7074 - c1_accuracy: 0.1911 - c2_accuracy: 0.1717 - c3_accuracy: 0.1801 - c4_accuracy: 0.1917

KeyboardInterrupt: 